# <center> Baseball Pitch Simulation <center>


#### <center> Preston Ito <center>


<img src="kershaw.jpg" width="700" />


When I was 10 years old pitching for the Pukalani Braves in Little League Baseball, I wanted so badly to be able to throw a curveball. I was able to throw an accurate fastball, but I wished to expand my pitching arsenal. My tiny little hands could barely cover half of the baseball, but I didn't let that discourage my pitching dreams. Unfortunately, even Google couldn't provide me with any answers after looking up, "How to throw a curveball for kids." I tried and tried but to no avail. My small body just didn't have what it took to throw a curveball, or any breaking pitch for that matter. 10 years later, I've decided to accomplish my childhood goals in another way: through computer code. 


## <center> How does it work? <center>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from math import cos,sin,pi,tan,radians
import vpython as vp
from finalmodule import makefield,bigplot,plot

ModuleNotFoundError: No module named 'vpython'

I've created a simulation that can throw fastballs, curveballs, and sliders at any user-input speed. First, I had to define all constants including the mass and radius of ball, gravity, density of air, cross-sectional area, initial spin and velocities in all 3 axes, throwing angle, and $k_d$ and $k_l$. $k_d$ and $k_l$ are the drag and lift coefficients that I use in all 3 ($x$,$y$ and $z$ directions) of the acceleration equations. When the cell below runs, it'll ask the user to choose a pitch by typing "FB", "CB", or "SL". FB is a fastball, CB is a curveball, and SL is a slider. Depending on what the user selects, the code assigns different values for spin, direction, and throwing angle. For example, the fastball is thrown downwards at an angle of $1.2 º$ with backspin of $200$ $\frac {rad}{s}$. The slider is thrown with a side spin of $-250$ $\frac {rad}{s}$. To account for the side spin, I've included an initial velocity in the opposite direction. Furthermore, this cell asks the user for an input of initial velocity. Typically, fastballs are thrown between 94-99 mph, sliders are thrown between 82-90 mph, and curveballs range from 75-85 mph.



*Note: This program has realistic numbers for spin to most accurately model real baseball pitches. However, if you've ever wanted to see what a fastball with unrealistically insane backspin or a slider that moves more than 4 feet side to side, you can change the values for the different omegas to see how it affects the graphs. Furthermore, I recommend inputting unrealistic velocities just to explore and see how the path of travel is affected.*

<img src="spin.png" width="250" />

In [2]:
#global constants 

cd = 0.4     #drag coefficient
rho = 1.2    #density of air
m = 0.1447     #mass of baseball in kg
rad = 0.07376/2       #radius of baseball in m
A = pi*rad**2      
omegax = 0      #none of these three types of pitches have spin on the x-axis
g = 9.8      #gravity
kd = cd*rho*A/(2*m)       #k_d constant used in diff eq
kl = rad*rho*A/(2*m)       #k_l constant used in diff eq

i = input("FB, CB, or SL")      #user chooses fastball, curveball, or slider
if i == "FB":
    omegay = -200        #fastballs have backspin of ~200 rad/s
    omegaz = 0
    v0y = 0
    theta = radians(-1.2)      #fastballs are thrown at ~-1.8 degrees
    
elif i == "CB":
    omegay = 315         #curveballs have topspin of ~315 rad/s
    omegaz = 0
    v0y = 0
    theta = radians(3)      #curveballs are thrown at ~3 degrees

elif i == "SL":          #this slider simulation is thrown by a RHP
    omegaz = -250        #sliders have side spin of ~250 degrees 
    omegay = 0
    v0y = 2             #thrown slightly to the right with a small initial velocity
    theta = radians(1.2)    #sliders are thrown at ~1.5 degrees
    
a = float(input("Velocity of pitch (mph)"))   #asks user to input initial velocity
v0 = a/2.237


FB, CB, or SL FB
Velocity of pitch (mph) 90


### <center> Solving differential equations <center>

Once I assigned all of my constants, I assigned different variables to the different numbers in the r array, which is an array of initial values of position and velocity in all 3 directions. I estimated a release height of 6ft for the ball. I used $sin$ and $cos$ to determine the initial velocities in the $x$ and $z$ directions. Then, I used 4th order Runge-Kutta to solve the different differential equations. I got all equations from *Computer Modeling: From Sports to Spaceflight...From Order to Chaos by J.M.A. Derby*. Essentially, these equations come from Newton's Second Law $\sum{F}=ma$, with 3 different forces: gravity, lift from the seams and spin of the ball, and drag from air resistance.

<img src="ballforce.gif" width="350" />

<center>$F_g = -mg$<center>    <center>$F_L = \frac{C_LρAv^2}{2}$<center>  <right>$F_D = -\frac{C_DρAv^2}{2}$<right>

<center> $dv_x = -vk_dv_x+(k_l(\omega_yv_z-\omega_zv_y))$ <center>
<center> $dv_y = -vk_dv_y+(k_l(\omega_zv_x-\omega_xv_z))$ <center>
<center> $dv_z = -vk_dv_z+(k_l(\omega_xv_y-\omega_yv_x))$ <center>    

*Note: The differential equations from the book were written so that $z$ is the vertical axis. This differs from the simulation in which $y$ is the vertical axis. This won't affect us yet, but it's something to keep in mind moving forward.*

In [5]:
def f(r):
    
    #assigning variables to the r array of different initial values
    x = r[0]
    y = r[1]
    z = r[2]
    vx = r[3]
    vy = r[4]
    vz = r[5]
    
    v = (vx**2+vy**2+vz**2)**0.5      #magnitude of velocity 
    
    #diff eqs from Kuchera book pg. 152
    dx = vx 
    dvx = -kd*v*vx+((kl*(omegay*vz-omegaz*vy)))
    dy = vy
    dvy = -kd*v*vy+((kl*(omegaz*vx-omegax*vz)))
    dz = vz
    dvz = -kd*v*vz+((kl*(omegax*vy-omegay*vx)))-g
    
    return np.array([dx,dy,dz,dvx,dvy,dvz],float)  #IMPORTANT: in the diff eqs given, the z-axis is the vertical axis. 
                                                    #However, in vpython, the y-axis is the vertical axis


### <center> Runge-Kutta <center>

Runge-Kutta is an ordinary differential equation solver that takes takes higher order derivatives to solve. Below are the equations for 4th order Runge-Kutta, where r represents the equations of motions in the $x$, $y$, and $z$ directions. The Runge-Kutta updates each position every 0.05 seconds until the z-position (height) is at 0. The cell below returns 3 lists: one for each direction. 





$$ k_1 = hf(r) $$
$$ k_2 = hf(r+\frac{1}{2}k_1) $$
$$ k_3 = hf(r+\frac{1}{2}k_2) $$
$$ k_4 = hf(r+k_3) $$
$$ r = r + \frac{1}{6}(k_1+2k_2+2k_3+k_4)$$

In [6]:
def runge(dt):

    #for plotting
    xs = [] 
    ys = []
    zs = []
    
    
    #taking the x and z components
    v0x = v0*cos(theta)         
    v0z = v0*sin(theta)
    
    r = np.array([0,0,1.8288,v0x,v0y,v0z])  #r array of initial values. starts at a height of ~6ft or ~1.8288m
    h = dt
    
    #4th order Runge-Kutta while ball is in air
    while r[2] >= 0:
        xs.append(r[0]) # calc x for plotting
        ys.append(r[1]) # calc y for plotting
        zs.append(r[2])
        k1 = h*f(r)
        k2 = h*f(r+0.5*k1)
        k3 = h*f(r+0.5*k2)
        k4 = h*f(r+k3)
        r += (k1+2*k2+2*k3+k4)/6
    return xs,ys,zs,v0            #return v0 for use later in vpython function


## <center> Plotting <center>

With the 3 lists from above, we can now plot them. The first plot function I've created plots the height against the distance to show the path of travel in two dimensions. Remember, I have to treat the $z$-direction as the traditional $y$-direction because of our differential equations. The second plot function called "bigplot" graphs the path of travel in three dimensions. Try graphing the slider! The 3D plot really shows how much side movement it has.

Aside from the two plots, I also created a vpython simulation to model the pitch with a strikezone. I enlarged the ball and the strikezone so the simulation looks nicer. The ball's position is updated with a for loop that goes through each item in all three lists. To make the ball travel in a speed proportional to the user input velocity, I made the rate at which the loop updates some factor times the user input. That way,  for example, a 99mph fastball will move across the screen faster than a 95mph. To move around in the vpython simulation, you can scroll up and down to zoom in and out. To move laterally, hold down shift while left clicking and move around. To change the angle, hold down control while left clicking.

*Note: Sometimes vpython won't work. If this happens, try restarting the kernel and trying again. If the problem persits, refresh the whole page.*



In [7]:
def main():
    
    xs,ys,zs,v0 = runge(0.01)   #time step of 0.01 seconds
    makefield(xs,ys,zs,v0)
    xf,yf,zf = plot(xs,ys,zs)
    bigplot(xf,yf,zf)
    
if __name__=="__main__":
    main()

NameError: name 'makefield' is not defined

## <center> Things I Had Trouble With <center>

There were a couple things I didn't account for in this simulation. One thing is that the coordinates of the ball when it's being released is not (0,0,6) like I said it was. There is almost always a $y$-component when the ball is being released. There are two reasons why I didn't incorporate this into my final project. First and foremost, I completely forgot about the $y$-position. I forgot that the pitcher doesn't release the ball directly inline with home plate. Secondly, by the time I realized I had left this out, I thought that it wouldn't add too much to the simulation since the simulations purpose is to show the movement of the ball with spin. Starting the ball at a position of $y=0$ is a better way to show the movement of the slider (which has side spin). Another thing to note is that a lot of assumptions were taken when writing this code. I took a rough average of spin from MLB pitchers for each pitches' spin. I also assumed the throwing angle of the different pitches, which again, was a rough estimate based on videos and simulation runs. 




#### <center>References<center>
    
    
    
https://baseballsavant.mlb.com/leaderboard/pitch-arsenals 
    
https://www.mlb.com/news/statcast-spin-rate-compared-to-velocity-c160896926
    
https://rapsodo.com/wp-content/uploads/2019/07/MLB-PitchingGuide.pdf
    
https://prospects365.com/2020/08/11/examining-pitching-approach-angles/
    
Title:    Computer modeling : from sports to spaceflight-- from order to chaos /
               Author:   Danby, J. M. A.
               Material: BOOK
    